<a href="https://colab.research.google.com/github/najeeb31x/Chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!pip install streamlit
import os
import nltk
import ssl
import streamlit as st
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath("nltk_data"))
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
intents = [
    {
      "tag": "fiction_books",
      "patterns": [
        "Can you recommend some good fiction books?",
        "Suggest some popular fiction novels.",
        "What are some must-read fiction books?"
      ],
      "responses": [
        "If you're into fiction, I recommend 'To Kill a Mockingbird' by Harper Lee, '1984' by George Orwell, or 'The Great Gatsby' by F. Scott Fitzgerald. They are timeless classics.",
        "'The Catcher in the Rye' by J.D. Salinger and 'Pride and Prejudice' by Jane Austen are excellent choices for fiction lovers.",
        "You can explore contemporary fiction like 'Where the Crawdads Sing' by Delia Owens or 'The Night Circus' by Erin Morgenstern."
      ]
    },
    {
      "tag": "nonfiction_books",
      "patterns": [
        "What are some good nonfiction books?",
        "Can you suggest a nonfiction book to read?",
        "Recommend a famous nonfiction book."
      ],
      "responses": [
        "'Sapiens: A Brief History of Humankind' by Yuval Noah Harari is an insightful read about human history.",
        "For self-improvement, try 'Atomic Habits' by James Clear or 'The Power of Habit' by Charles Duhigg.",
        "Interested in memoirs? 'Becoming' by Michelle Obama and 'Educated' by Tara Westover are highly recommended."
      ]
    },
    {
      "tag": "mystery_books",
      "patterns": [
        "What are the best mystery novels?",
        "Recommend some good detective stories.",
        "Can you suggest thrilling mystery books?"
      ],
      "responses": [
        "You might enjoy 'Gone Girl' by Gillian Flynn, a psychological thriller full of twists.",
        "'The Girl with the Dragon Tattoo' by Stieg Larsson is a gripping mystery that will keep you hooked.",
        "For classic detective stories, read 'The Hound of the Baskervilles' by Arthur Conan Doyle or 'And Then There Were None' by Agatha Christie."
      ]
    },
    {
      "tag": "fantasy_books",
      "patterns": [
        "Suggest some good fantasy novels.",
        "What are the best fantasy books to read?",
        "Recommend an epic fantasy book."
      ],
      "responses": [
        "'The Lord of the Rings' by J.R.R. Tolkien is a masterpiece of epic fantasy.",
        "For something modern, try 'A Song of Ice and Fire' series by George R.R. Martin or 'The Name of the Wind' by Patrick Rothfuss.",
        "Young adult fantasy fans will love 'Harry Potter' by J.K. Rowling or 'Percy Jackson' by Rick Riordan."
      ]
    },
    {
      "tag": "science_fiction_books",
      "patterns": [
        "What are the best sci-fi books?",
        "Recommend a good science fiction novel.",
        "Can you suggest some popular sci-fi books?"
      ],
      "responses": [
        "'Dune' by Frank Herbert is a science fiction classic filled with political intrigue and epic world-building.",
        "'Ender's Game' by Orson Scott Card and 'The Martian' by Andy Weir are great for fans of futuristic tales.",
        "For dystopian sci-fi, consider 'Brave New World' by Aldous Huxley or 'Fahrenheit 451' by Ray Bradbury."
      ]
    },
    {
      "tag": "self_help_books",
      "patterns": [
        "What are some good self-help books?",
        "Recommend books for personal growth.",
        "Can you suggest motivational books?"
      ],
      "responses": [
        "'The 7 Habits of Highly Effective People' by Stephen Covey is a transformative book for personal and professional success.",
        "'How to Win Friends and Influence People' by Dale Carnegie is a timeless guide to improving social skills.",
        "For mindset shifts, read 'The Subtle Art of Not Giving a F*ck' by Mark Manson or 'Think and Grow Rich' by Napoleon Hill."
      ]
    },
      {
      "tag": "greeting",
      "patterns": [
          "Hi",
          "Hello",
          "Hey",
          "How are you",
          "What's up"
      ],
      "responses": [
          "Hi there",
          "Hello",
          "Hey",
          "I'm fine, thank you",
          "Nothing much"
      ]
  },
  {
      "tag": "goodbye",
      "patterns": [
          "Bye",
          "See you later",
          "Goodbye",
          "Take care"
      ],
      "responses": [
          "Goodbye",
          "See you later",
          "Take care"
      ]
  },
  {
      "tag": "thanks",
      "patterns": [
          "Thank you",
          "Thanks",
          "Thanks a lot",
          "I appreciate it"
      ],
      "responses": [
          "You're welcome",
          "No problem",
          "Glad I could help"
      ]
  },
  {
      "tag": "about",
      "patterns": [
          "What can you do",
          "Who are you",
          "What are you",
          "What is your purpose"
      ],
      "responses": [
          "I am a chatbot",
          "My purpose is to assist you",
          "I can answer questions and provide assistance"
      ]
  },
  {
      "tag": "help",
      "patterns": [
          "Help",
          "I need help",
          "Can you help me",
          "What should I do"
      ],
      "responses": [
          "Sure, what do you need help with?",
          "I'm here to help. What's the problem?",
          "How can I assist you?"
      ]
  },
  {
      "tag": "age",
      "patterns": [
          "How old are you",
          "What's your age"
      ],
      "responses": [
          "I don't have an age. I'm a chatbot.",
          "I was just born in the digital world.",
          "Age is just a number for me."
      ]
  },
    {
      "tag": "children_books",
      "patterns": [
        "What are some good books for kids?",
        "Recommend books for young readers.",
        "Can you suggest children's storybooks?"
      ],
      "responses": [
        "For younger kids, try 'The Very Hungry Caterpillar' by Eric Carle or 'Goodnight Moon' by Margaret Wise Brown.",
        "For older children, classics like 'Charlotte's Web' by E.B. White and 'The Chronicles of Narnia' by C.S. Lewis are perfect.",
        "'Harry Potter' by J.K. Rowling is a great series that bridges the gap between children and young adult fiction."
      ]
    }
  ]

In [4]:
# Create the vectorizer and classifier
vectorizer = TfidfVectorizer()
clf = LogisticRegression(random_state=0, max_iter=10000)

# Preprocess the data
tags = []
patterns = []
for intent in intents:
    for pattern in intent['patterns']:
        tags.append(intent['tag'])
        patterns.append(pattern)

# training the model
x = vectorizer.fit_transform(patterns)
y = tags
clf.fit(x, y)


LogisticRegression(max_iter=10000, random_state=0)

In [5]:
def chatbot(input_text):
    input_text = vectorizer.transform([input_text])
    tag = clf.predict(input_text)[0]
    for intent in intents:
        if intent['tag'] == tag:
            response = random.choice(intent['responses'])
            return response

In [6]:
counter = 0

def main():
    global counter
    st.title("Chatbot")
    st.write("Welcome to the chatbot. Please type a message and press Enter to start the conversation.")

    counter += 1
    user_input = st.text_input("You:", key=f"user_input_{counter}")

    if user_input:
        response = chatbot(user_input)
        st.text_area("Chatbot:", value=response, height=100, max_chars=None, key=f"chatbot_response_{counter}")

        if response.lower() in ['goodbye', 'bye']:
            st.write("Thank you for chatting with me. Have a great day!")
            st.stop()

if __name__ == '__main__':
    main()


2024-12-09 14:17:59.929 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-09 14:18:00.444 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-12-09 14:18:00.449 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-09 14:18:00.453 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-09 14:18:00.458 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-09 14:18:00.462 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-09 14:18:00.481 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-09 14:18:00.486 Thread 'MainThread': mi